<a href="https://colab.research.google.com/github/lastinm/ml_hw_notebooks/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%961_Apache_Spark_(MapReduce).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подключение Google диска

 Для работы с текстовым файлом подключим google диск.

In [1]:
 # Подключаем Google Drive
 from google.colab import drive
 drive.mount('/content/drive')

Mounted at /content/drive


# Задание 1

Используя Apache Spark (MapReduce), реализовать программу для подсчета количества слов в тексте, который представляет собой некоторое количество строк, единственный разделитель между словами – пробел.

## Решение

*MapReduce в контексте Spark*

**MapReduce** — это модель программирования, которая изначально была предложена Google для обработки и генерации больших наборов данных. Она разделяет задачу обработки на две основные фазы:

**Map**: Данные обрабатываются функцией Map, которая берет набор данных и преобразует его в набор пар "ключ-значение".
**Reduce**: Затем функция Reduce принимает эти пары ключей и значений, группирует их и производит итоговые результаты.

### Загрузка данных

Есть текстовый файл со сказкой А.С. Пушкина "Золотая рыбка", содержащий строки текста. Мы будем загружать содержимое этого файла в RDD.

In [ ]:
# Импортируем необходимые библиотеки
from pyspark import SparkConf, SparkContext

In [ ]:
# Загружаем текстовый файл
text_file = sc.textFile("/content/drive/MyDrive/Data/Пушкин А.С. - Сказка о рыбаке и рыбке.txt")

### Инициализация SparkContext

Инициализируем контекст Spark. Это основа для работы с RDD (Resilient Distributed Dataset).

In [ ]:
# Создаем конфигурацию Spark
conf = SparkConf().setAppName("WordCount").setMaster("local[*]")
sc = SparkContext(conf = conf)

### Подсчет слов

Для понимания, как работает Spark, рассмотрим простой пример подсчета слов, который можно реализовать с использованием концепций MapReduce.

Map: Преобразуем текст в слова, создавая пары "слово, 1".

Reduce: Суммируем значения для каждого слова, чтобы получить его общее количество.

In [ ]:
# Преобразуем текст в слова
words = text_file.flatMap(lambda line: line.split(" "))

# Преобразуем каждую строку строку со словом в пару "слово" и цифра 1
list_words = words.map(lambda word: (word, 1))
list_words.take(5)

[('Жил', 1), ('старик', 1), ('со', 1), ('своею', 1), ('старухой', 1)]

In [ ]:
# Подсчитываем количество слов
word_counts = list_words.reduceByKey(lambda a, b: a + b)
word_counts.take(10)

[('Жил', 1),
 ('старухой', 1),
 ('У', 2),
 ('самого', 1),
 ('синего', 1),
 ('моря;', 1),
 ('Они', 1),
 ('землянке', 1),
 ('Ровно', 1),
 ('тридцать', 2)]

# Задание 2

Используя Apache Spark (DataFrame), реализовать программу для подсчета количества слов в тексте, которые представляет собой некоторое количество строк, единственный разделитель между словами – пробел.

## Решение

**DataFrame** в **Apache Spark** — это распределенная коллекция данных, организованных в виде таблицы. Он аналогичен таблицам в реляционных базах данных и предоставляет возможность выполнять операции SQL.

### Загрузка данных

Есть текстовый файл со сказкой А.С. Пушкина "Золотая рыбка", содержащий строки текста. Мы будем загружать содержимое этого файла в DataFrame.

In [7]:
# Импорт необходимых библиотек
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col

# Создание SparkSession
spark = SparkSession.builder \
    .appName("Загрузка текстового файла в DataFrame") \
    .getOrCreate()

In [8]:
# Загрузка текстового файла в DataFrame
# При загрузке текстового файла каждая строка файла будет представлена как отдельная запись
df = spark.read.text('/content/drive/MyDrive/Data/Пушкин А.С. - Сказка о рыбаке и рыбке.txt')
df.show(5)

+--------------------+
|               value|
+--------------------+
|Жил старик со сво...|
|У самого синего м...|
|Они жили в ветхой...|
|Ровно тридцать ле...|
|Старик ловил нево...|
+--------------------+
only showing top 5 rows



### Подсчет слов

In [11]:
# Разделение строк на слова, используя пробел как разделитель
# explode создает отдельные строки для каждого слова
words_df = df.select(explode(split(col("value"), " ")).alias("word"))
words_df.show(5)

+--------+
|    word|
+--------+
|     Жил|
|  старик|
|      со|
|   своею|
|старухой|
+--------+
only showing top 5 rows



In [12]:
# Считать вхождения каждого слова
word_count_df = words_df.groupBy("word").count()

# Показать результат
print("Количество вхождений каждого слова:")
word_count_df.show()

Количество вхождений каждого слова:
+-----------+-----+
|       word|count|
+-----------+-----+
|       себя|    1|
|     баба».|    1|
|         Уж|    3|
|     теперь|    2|
|       чтоб|    1|
|     просит|    1|
|   душенька|    2|
|   старуху,|    1|
|     плечах|    1|
|          —|    8|
|прикрикнула|    1|
|    невежа!|    1|
|    поперек|    1|
|        так|    1|
|       Чай,|    1|
|    сказала|    1|
|    неделя,|    2|
|    умеешь,|    1|
|   молвить.|    1|
|          У|    2|
+-----------+-----+
only showing top 20 rows



# Задание 3

Доработайте программу из задания 2: учитывайте, что одно слово может быть записано в разном регистре (Word, word, WORD – одно слово), в тексте могут присутствовать знаки пунктуации, а также то, что количество пробелов между словами может быть больше одного.

## Решение

## Импорт дополнительных модулей

In [13]:
from pyspark.sql.functions import lower, regexp_replace, count, desc

## Предобработка исходного датафрейма

In [47]:
# Нормализация текста:
# - Приведение к нижнему регистру
# - Удаление знаков пунктуации с помощью регулярных выражений
# Регулярное выражение без учета дефиса и частиц
#normalized_df = df.select(regexp_replace(lower(col("value")), "[^а-яА-ЯёЁ\\s]", "").alias("normalized_value"))
# Учли, что слова могут содержать дефис для отделения частиц
normalized_df = df.select(regexp_replace(lower(col("value")), "[^a-zA-Zа-яА-ЯёЁ\\s-]", "").alias("normalized_value"))

normalized_df.show(5)

+--------------------+
|    normalized_value|
+--------------------+
|жил старик со сво...|
|у самого синего моря|
|они жили в ветхой...|
|ровно тридцать ле...|
|старик ловил нево...|
+--------------------+
only showing top 5 rows



In [48]:
# Разделение строк на слова, учитывая произвольное количество пробелов
words_df = normalized_df.select(explode(split(col("normalized_value"), "\\s+")).alias("word"))
words_df.show(5)

+--------+
|    word|
+--------+
|     жил|
|  старик|
|      со|
|   своею|
|старухой|
+--------+
only showing top 5 rows



In [49]:
# Удаление пустых строк (если они остались)
words_df = words_df.filter(col("word") != "")

In [50]:
# Подсчет вхождений каждого слова
word_count_df = words_df.groupBy("word").count()
word_count_df.show()

+-----------+-----+
|       word|count|
+-----------+-----+
|       себя|    1|
|    простую|    1|
|   выпросил|    2|
|  насмешишь|    1|
|     теперь|    2|
|      тиной|    1|
|       чтоб|    2|
|     просит|    1|
|   душенька|    2|
|     плечах|    1|
|прикрикнула|    1|
|        так|    5|
|    поперек|    1|
|    сказала|    1|
|раскололось|    2|
|    поделом|    1|
|      пряла|    1|
|     третий|    1|
|      хочет|    6|
|     служат|    1|
+-----------+-----+
only showing top 20 rows



## Топ-10 самых часто встречаемых слов

In [51]:
# Топ-10 самых часто встречаемых слов
top_10_words = word_count_df.orderBy(desc("count")).limit(10)
top_10_words.show()


+-------+-----+
|   word|count|
+-------+-----+
|     не|   29|
|      с|   21|
|  рыбка|   19|
|      в|   17|
|старуха|   17|
|     он|   17|
|      к|   16|
| старик|   16|
|     на|   14|
|      и|   14|
+-------+-----+



## Слово, которое встречается чаще всего

In [52]:
# Слово, которое встречается чаще всего
most_frequent_word = word_count_df.orderBy(desc("count")).first()
most_frequent_word

Row(word='не', count=29)

## Слово, которое встречается реже всего

In [53]:
# Слово, которое встречается реже всего
least_frequent_word = word_count_df.orderBy("count").first()
least_frequent_word

Row(word='себя', count=1)

## Средняя встречаемость слов в тексте

In [54]:
# Средняя встречаемость слов
average_frequency = word_count_df.agg({"count": "avg"}).first()[0]
average_frequency

2.4205128205128204

## Общее число уникальных слов

In [55]:
# Общее число уникальных слов
unique_word_count = word_count_df.count()
unique_word_count

390

## Общее число слов в тексте

In [56]:
# Общее количество слов в тексте
total_word_count = words_df.count()
total_word_count

944